## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [86]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [87]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as error:
    print(error)

#### Create & Set Keyspace

In [88]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosqldb 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as error:
    print(error)

try:
    session.set_keyspace('nosqldb')
except Exception as error:
    print(error)

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [89]:
query = "CREATE TABLE IF NOT EXISTS music"
query = query + "(artist_name text, song_title text ,length text, session_no text ,item_in_session text, PRIMARY KEY (session_no ,artist_name))"
try:
    session.execute(query)
except Exception as error:
    print(error)

### Let's insert our data into of table

In [90]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) 
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO music(artist_name,song_title,length,session_no,item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[0], row[9], row[5], row[8], row[3]))

### Validate our Data Model using a SELECT

In [92]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "select artist_name,song_title, length from music where session_no = '338' AND item_in_session = '4' ALLOW FILTERING"
rows=''
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
    
for row in rows:
    print ("Artist:",row.artist_name,"  ","Song Title:",row.song_title," ","Length",row.length)

Artist: Faithless    Song Title: Music Matters (Mark Knight Dub)   Length 495.3073


In [93]:
# q = "drop table music"
# session.execute(q)

### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [76]:

query = "CREATE TABLE IF NOT EXISTS music1"
query = query + "(user_id text, artist_name text, song_title text , name text, session_no text ,item_in_session text, PRIMARY KEY ((user_id,session_no),item_in_session)) WITH CLUSTERING ORDER BY (item_in_session ASC)"
try:
    session.execute(query)
except Exception as error:
    print(error)


### Let's insert our data into of table

In [77]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO music1(user_id,artist_name,song_title,name,session_no,item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s,  %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[10], row[0],row[9], row[1]+row[4], row[8], row[3]))

### Validate our Data Model using a SELECT

In [78]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "select artist_name,song_title,name from music1 where user_id = '10' AND session_no = '182' ALLOW FILTERING"
rows=''
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
    
for row in rows:
    print ("Artist:",row.artist_name,"  ","Song Title:",row.song_title," ","Name:",row.name)

Artist: Three Drives    Song Title: Greece 2000   Name: SylvieCruz
Artist: Lonnie Gordon    Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)   Name: SylvieCruz
Artist: Down To The Bone    Song Title: Keep On Keepin' On   Name: SylvieCruz
Artist: Sebastien Tellier    Song Title: Kilometer   Name: SylvieCruz


In [79]:
# q = "drop table music1"
# session.execute(q)

### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [80]:
query = "CREATE TABLE IF NOT EXISTS music2"
query = query + "(fname text,lname text, song_title text , user_id text, PRIMARY KEY (song_title,user_id))"
try:
    session.execute(query)
except Exception as error:
    print(error)

### Let's insert our data into of table

In [81]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = "INSERT INTO music2(fname,lname,song_title,user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[1], row[4], row[9],row[10]))

### Validate our Data Model using a SELECT

In [82]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
query = "select fname,lname  from music2 where song_title = 'All Hands Against His Own' ALLOW FILTERING"
rows=''
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
    
for row in rows:
    print (row.fname,' ',row.lname)

Jacqueline   Lynch
Tegan   Levine
Sara   Johnson


### Drop the tables before closing out the sessions

In [83]:
# q = "drop table music2"
# session.execute(q)

### Close the session and cluster connection¶

In [84]:
session.shutdown()
cluster.shutdown()